In [95]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

openjdk-8-jdk-headless is already the newest version (8u242-b08-0ubuntu3~18.04).
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.


In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [98]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
%cd '/content/drive/My Drive/XLDLL/Assignment2_Ex'

/content/drive/My Drive/XLDLL/Assignment2_Ex


In [0]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
import pandas as pd

# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

In [0]:
def recList(pair):
	person = pair[0]
	cntfList = pair[1]
	cntfLen = len(cntfList)
	rlist = []
	for i in range(cntfLen):
		for j in range(len(cntfList[i][1])):
			rlist.append(cntfList[i][1][j])
	return (person, rlist)

In [0]:
def friend_pairs_of_user(user_friendslist): #bat tung cap 1
	from_user = user_friendslist[0]
	friendLst = user_friendslist[1]
	return [(pair_of_usersFriend, 1) for pair_of_usersFriend in it.combinations(friendLst, 2)] #tra ve tung cap to hop 1 trong friendList

In [0]:
def mergeByShareCnt(x, y):
	for i in y:
		if i in x:
			x[i] += y[i]
		else:
			x[i] = y[i]
	return x
  

In [0]:
def sortRecommendsByCnt(recommendTuple):
	rlist = []
	rdict = recommendTuple[1]
	ckeys = sorted(rdict, reverse=True)
	for cnt in ckeys:
	    rlist.append((cnt, sorted(rdict[cnt])))
	return (recommendTuple[0], rlist)



In [0]:
import re
import sys
import itertools as it
from pyspark import SparkConf, SparkContext


friendLines = sc.textFile('/content/drive/My Drive/XLDLL/Assignment2_Ex/soc-LiveJournal1Adj.txt')





In [0]:
user_friends = friendLines.map(lambda l: l.split())

user  = user_friends.map(lambda x: x[0])  #cat cac user ra

Users = user.collect() #map lai thanh 1 list
Users = map(int, Users) #convert qua list ->list tat ca Users


In [0]:
user_w_friends = user_friends.filter(lambda e: len(e) == 2) #loc ra nhung user khong co tren 2 nguoi ban
user_friendList = user_w_friends.map(lambda p: (int(p[0]), map(int, sorted(p[1].split(','))))) #Map tung user voi friend va sap xep mang friend de cac buoc sau ghep cap lai
user_friendList.take(1)

In [0]:
pairs_from_commonFriend = user_friendList.map(friend_pairs_of_user).flatMap(lambda x: x)  # EX: [[(1,10),1...][(2,1),1]] - >[((1,10),1),((2,1),1)]
pairs_from_commonFriend.take(1000)

In [0]:
pairs_CntOf_commonFriends = pairs_from_commonFriend.reduceByKey(lambda c1, c2: c1 + c2) # cộng gộp các cặp giá trị lại với nhau
pairs_CntOf_commonFriends.take(5)

[((11577, 19689), 2),
 ((41739, 4875), 1),
 ((7269, 8749), 1),
 ((41106, 45594), 2),
 ((46668, 46724), 21)]

In [0]:
user2friendsMap = user_friendList.collectAsMap() #collect nhu 1 dictionary
user2friendsMap[0]

In [0]:
pairs_hasCommon_yetFriends = pairs_CntOf_commonFriends.filter(lambda pC: pC[0][1] not in user2friendsMap[pC[0][0]])   # tra ve danh sach cac cap co ban chung nhung chua ket ban
pairs_hasCommon_yetFriends.take(5)

[((14207, 23531), 2),
 ((24202, 6142), 1),
 ((27680, 29508), 2),
 ((31783, 31811), 1),
 ((1445, 39473), 2)]

In [0]:
user_recommendList_byShareCnt = pairs_hasCommon_yetFriends.map(lambda pC: [(pC[0][0], {pC[1]: [pC[0][1]]}), (pC[0][1], {pC[1]: [pC[0][0]]})]).flatMap(lambda x: x) #tach ra theo key
user_recommendList_byShareCnt.take(5)

[(49405, {1: [615]}),
 (615, {1: [49405]}),
 (10759, {1: [41775]}),
 (41775, {1: [10759]}),
 (17999, {3: [24345]})]

In [0]:
recommendList_byShareCnt = user_recommendList_byShareCnt.reduceByKey(mergeByShareCnt) # link cac gap key thanh list dai` co lien ket voi nhau de recommend
recommendList_byShareCnt.take(5)

In [0]:
recommends = recommendList_byShareCnt.map(sortRecommendsByCnt) #sap xep lai list recommend

In [0]:
recommendList = recommends.map(recList)

In [0]:
recommendMap = recommendList.collectAsMap()

In [106]:
List = [924, 8941, 8942, 9019, 9020, 9021, 9022, 9990, 9992, 9993]
for i in List:
  print str(i) + "\t" + str(recommendMap[i])

924	[439, 2409, 6995, 11860, 15416, 43748, 45881]
8941	[8943, 8944, 8940]
8942	[8939, 8940, 8943, 8944]
9019	[9022, 317, 9023]
9020	[9021, 9016, 9017, 9022, 317, 9023]
9021	[9020, 9016, 9017, 9022, 317, 9023]
9022	[9019, 9020, 9021, 317, 9016, 9017, 9023]
9990	[13134, 13478, 13877, 34299, 34485, 34642, 37941]
9992	[9987, 9989, 35667, 9991]
9993	[9991, 13134, 13478, 13877, 34299, 34485, 34642, 37941]


In [0]:
# HERE

Once you obtained the desired results, **head over to Gradescope and submit your solution for this Colab**!